# ENSF 544 Data Science for Software Engineers
## Phase 1
#### By: Kyle Friedt, Erslan Salman, Kelvin Tran and Avneet Gill

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

def get_addresses(url):
    addresses = []
    postalcodes = []
    
    zomato_html = get_page_html(url)
    parser = BeautifulSoup(zomato_html, 'html.parser')
    address_tag = parser.find_all("div", class_ = "col-m-16 search-result-address grey-text nowrap ln22")
    
    for address in address_tag:
        postal = None
        temp  = address.get_text()
        temp = temp.strip("\n")
        temp = temp.strip()
        
        if re.search('[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d$',temp) != None:
            postal = re.findall('[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d$',temp)[0]
            temp = temp.replace(postal,"")

        elif re.search('[A-Za-z]\d[A-Za-z]$',temp) != None:
            postal = re.findall('[A-Za-z]\d[A-Za-z]',temp)[0]
            temp = temp.replace(postal,"")
            
        addresses.append(temp)
        postalcodes.append(postal)
 
    return addresses, postalcodes
    
def get_titles(url):
    titles = []

    zomato_html = get_page_html(url)
    parser = BeautifulSoup(zomato_html, 'html.parser')
    
    title_tag = parser.find_all("a", class_ = "result-title" )
    
    for title in title_tag:
        temp  = title.get_text()
        temp = temp.strip("\n")
        temp = temp.strip()
        titles.append(temp)
    
    return titles
    
def get_next_page(parser):
    links_array = []
    
    next_page = parser.find("a", class_="paginator_item next item")
    next_page = "https://www.zomato.com" + str(next_page['href'])
    next_page = next_page.split("=")
    stripped = next_page[0]
    return stripped

def get_number_of_pages(parser):
    next_page = parser.find("div", class_="col-l-4 mtop pagination-number")
    next_page = next_page.find_all('b')
    number = next_page[1].get_text()
    return number 

def get_all_links(url,number):
    links = []
    links.append(url)
    url_ = url + "="
    
    for x in range(2,int(number)+1):
        new_link = url_+str(x)
        links.append(new_link)
        
    return links
    

def get_page_html(url):    
    headers = {"User-Agent":"Mozilla/5.0"}
    source=requests.get(url, headers=headers).text
    return source


def get_resturant_data(parser):
    url = get_next_page(parser)
    number = get_number_of_pages(parser)
    links = get_all_links(url, number)
    titles = []
    addresses = []
    postal_codes = []
    for link in links:
        title = get_titles(link)
        titles = titles + title
        address, postal = get_addresses(link)
        addresses.extend(address)
        postal_codes.extend(postal)
        
        print("loading...")
    
    df = pd.DataFrame(titles, columns = ['name'])
    df['address'] = addresses
    df['postal_code'] = postal_codes
    return df

url = "https://www.zomato.com/edmonton/downtown-restaurants?page=1"
zomato_html = get_page_html(url)
parser = BeautifulSoup(zomato_html, 'html.parser')

get_resturant_data(parser)

loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...

,name,address,postal_code
0,Tres Carnales Taqueria,"10119 100A Street, Edmonton",None
1,Corso 32,"10345 Jasper Avenue, Edmonton",T5J1Y7
2,Pampa Brazilian Steakhouse,"9929 109 St NW, Edmonton",T5K 1H6
3,Sabor Restaurant,"10220-103 Street, Edmonton",T5J0Y8
4,Sofra,10345 106th Street NW,T5J 0J2
...,...,...,...
2060,Bloom & Bliss Flower and Coffee Shop,"5115 50 St, Evansburg",T0E0T0
2061,Lily's Steak & Pizza,"4803 Queen St, Bruderheim",T0B0S0
2062,H&D Burger Barn,"4620 50 St, Millet",T0C1Z0
2063,Evansburg Snackbar,"4918 50th Ave, Evansburg",T0A3A0
